In [1]:
!pip install "sentence-transformers>=2.2.2" ray[train] torch torchvision accelerate mlflow psycopg2-binary pandas sqlalchemy numpy tqdm datasets


In [15]:

import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from sqlalchemy import create_engine, text
import pandas as pd
import tarfile
import shutil
import re
import unicodedata
from tqdm import tqdm
import mlflow

os.environ["MLFLOW_TRACKING_URI"] = "http://129.114.27.112:8000"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://129.114.27.112:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "admin"

os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin@123"
os.environ["MLFLOW_TRACKING_USERNAME"] = "admin"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "password"
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

#mlflow.set_experiment("arxiv-bi-encoder")

from datasets import Dataset

from datasets import Dataset
import os
import json
import logging
import pandas as pd
import mlflow
from datetime import datetime
from sqlalchemy import create_engine
from sentence_transformers import SentenceTransformer, InputExample, losses, models, LoggingHandler, util
from torch.utils.data import DataLoader
import torch


os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

#os.environ["MLFLOW_TRACKING_URI"] = "http://192.5.87.131:30003"
#os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://192.5.87.131:30000"
#os.environ["AWS_ACCESS_KEY_ID"] = "admin"
#os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin@123"
#os.environ["MLFLOW_TRACKING_USERNAME"] = "admin"
#os.environ["MLFLOW_TRACKING_PASSWORD"] = "password"
#mlflow.set_tracking_uri("http://192.5.87.131:30003")
#mlflow.set_experiment("arxiv-bi-encoder")

logging.basicConfig(format="%(asctime)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO,
                    handlers=[LoggingHandler()])


mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
#mlflow.set_experiment("distilbert-arxiv-bi-encoder1")

logging.basicConfig(format="%(asctime)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

df = pd.read_csv("arxiv_chunks_training_4_phrases1.csv").head(200)

train_examples = []
for _, row in df.iterrows():
    try:
        queries = json.loads(row['query']) if isinstance(row['query'], str) else row['query']
        if isinstance(queries, list):
            for q in queries:
                if isinstance(q, str) and len(q.strip()) > 0:
                    train_examples.append(InputExample(texts=[q.strip(), row['chunk_data']]))
        elif isinstance(queries, str):
            if len(queries.strip()) > 0:
                train_examples.append(InputExample(texts=[queries.strip(), row['chunk_data']]))
    except Exception as e:
        print(f"Skipping row due to error: {e}")


model_name = 'distilbert-base-uncased'
max_seq_length = 512
batch_size = 1
epochs = 1
model_save_path = f'output/arxiv-model-{datetime.now().strftime("%Y%m%d-%H%M%S")}'
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=True)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)

with mlflow.start_run():
    mlflow.log_params({
        "model_name": model_name,
        "train_batch_size": batch_size,
        "max_seq_length": max_seq_length,
        "epochs": epochs,
        "optimizer": "AdamW",
        "pooling": "mean"
    })

    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=epochs,
        warmup_steps=int(0.1 * len(train_dataloader)),
        show_progress_bar=True,
        use_amp=True,
        optimizer_params={"lr": 2e-5},
        checkpoint_path=model_save_path,
        checkpoint_save_steps=len(train_dataloader)
    )

    model.save(model_save_path)
    mlflow.log_artifacts(model_save_path, artifact_path="model")
    mlflow.sentence_transformers.log_model(model, artifact_path="model", registered_model_name="distilbert-arxiv-bi-encoder1")

print(f" Training complete. Model saved to: {model_save_path}")

print(os.environ.get("MLFLOW_TRACKING_URI"))



2025-05-09 21:51:54 - Use pytorch device_name: cpu


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.000000


2025-05-09 22:05:45 - Saving model checkpoint to output/arxiv-model-20250509-215154/checkpoint-600
2025-05-09 22:05:45 - Save model to output/arxiv-model-20250509-215154/checkpoint-600
2025-05-09 22:05:47 - Save model to output/arxiv-model-20250509-215154
2025-05-09 22:06:25 - Save model to /tmp/tmpu_mce60e/model/model.sentence_transformer


2025/05/09 22:06:25 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/09 22:06:49 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.20.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torchvision==0.20.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Successfully registered model 'distilbert-arxiv-bi-encoder1'.
2025/05/09 22:07:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: distilbert-arxiv-bi-encoder1, versi

🏃 View run carefree-wren-174 at: http://129.114.27.112:8000/#/experiments/2/runs/d1494deb427848008521c0611e6eecd4
🧪 View experiment at: http://129.114.27.112:8000/#/experiments/2
 Training complete. Model saved to: output/arxiv-model-20250509-215154
http://129.114.27.112:8000


In [16]:

import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from sqlalchemy import create_engine, text
import pandas as pd
import tarfile
import shutil
import re
import unicodedata
from tqdm import tqdm
import mlflow

os.environ["MLFLOW_TRACKING_URI"] = "http://129.114.27.112:8000"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://129.114.27.112:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "admin"

os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin@123"
os.environ["MLFLOW_TRACKING_USERNAME"] = "admin"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "password"
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

#mlflow.set_experiment("arxiv-bi-encoder")

from datasets import Dataset

from datasets import Dataset
import os
import json
import logging
import pandas as pd
import mlflow
from datetime import datetime
from sqlalchemy import create_engine
from sentence_transformers import SentenceTransformer, InputExample, losses, models, LoggingHandler, util
from torch.utils.data import DataLoader
import torch


os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

#os.environ["MLFLOW_TRACKING_URI"] = "http://192.5.87.131:30003"
#os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://192.5.87.131:30000"
#os.environ["AWS_ACCESS_KEY_ID"] = "admin"
#os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin@123"
#os.environ["MLFLOW_TRACKING_USERNAME"] = "admin"
#os.environ["MLFLOW_TRACKING_PASSWORD"] = "password"
#mlflow.set_tracking_uri("http://192.5.87.131:30003")
#mlflow.set_experiment("arxiv-bi-encoder")

logging.basicConfig(format="%(asctime)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO,
                    handlers=[LoggingHandler()])


mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
#mlflow.set_experiment("distilbert-arxiv-bi-encoder1")

logging.basicConfig(format="%(asctime)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

df = pd.read_csv("arxiv_chunks_training_4_phrases1.csv").head(380)

train_examples = []
for _, row in df.iterrows():
    try:
        queries = json.loads(row['query']) if isinstance(row['query'], str) else row['query']
        if isinstance(queries, list):
            for q in queries:
                if isinstance(q, str) and len(q.strip()) > 0:
                    train_examples.append(InputExample(texts=[q.strip(), row['chunk_data']]))
        elif isinstance(queries, str):
            if len(queries.strip()) > 0:
                train_examples.append(InputExample(texts=[queries.strip(), row['chunk_data']]))
    except Exception as e:
        print(f"Skipping row due to error: {e}")


model_name = 'distilbert-base-uncased'
max_seq_length = 512
batch_size = 2
epochs = 1
model_save_path = f'output/arxiv-model-{datetime.now().strftime("%Y%m%d-%H%M%S")}'
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=True)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)

with mlflow.start_run():
    mlflow.log_params({
        "model_name": model_name,
        "train_batch_size": batch_size,
        "max_seq_length": max_seq_length,
        "epochs": epochs,
        "optimizer": "AdamW",
        "pooling": "mean"
    })

    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=epochs,
        warmup_steps=int(0.1 * len(train_dataloader)),
        show_progress_bar=True,
        use_amp=True,
        optimizer_params={"lr": 2e-5},
        checkpoint_path=model_save_path,
        checkpoint_save_steps=len(train_dataloader)
    )

    model.save(model_save_path)
    mlflow.log_artifacts(model_save_path, artifact_path="model")
    mlflow.sentence_transformers.log_model(model, artifact_path="model", registered_model_name="distilbert-arxiv-bi-encoder1")

print(f" Training complete. Model saved to: {model_save_path}")

print(os.environ.get("MLFLOW_TRACKING_URI"))



2025-05-09 22:12:27 - Use pytorch device_name: cpu


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.130100


2025-05-09 22:32:41 - Saving model checkpoint to output/arxiv-model-20250509-221226/checkpoint-570
2025-05-09 22:32:41 - Save model to output/arxiv-model-20250509-221226/checkpoint-570
2025-05-09 22:32:43 - Save model to output/arxiv-model-20250509-221226
2025-05-09 22:33:20 - Save model to /tmp/tmpbzjsk8pa/model/model.sentence_transformer


2025/05/09 22:33:21 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/09 22:33:34 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.20.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torchvision==0.20.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Registered model 'distilbert-arxiv-bi-encoder1' already exists. Creating a new version of this model...
2025/05/09 22:33:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model

🏃 View run vaunted-shark-785 at: http://129.114.27.112:8000/#/experiments/2/runs/a10422d7f2f14158b94aa2b76d57188b
🧪 View experiment at: http://129.114.27.112:8000/#/experiments/2
 Training complete. Model saved to: output/arxiv-model-20250509-221226
http://129.114.27.112:8000
